In [0]:
# Run in python console
import nltk; nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
# Run in terminal or command prompt
!python3 -m spacy download en


    Linking successful
    /usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
    /usr/local/lib/python3.6/dist-packages/spacy/data/en

    You can now load the model via spacy.load('en')



In [0]:
!pip install pyLDAvis

    100% |████████████████████████████████| 1.6MB 7.1MB/s 
  Running setup.py bdist_wheel for pyLDAvis ... - \ done
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
Successfully built pyLDAvis


In [0]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [0]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [0]:
# read data
input_review_file = "Amazon_Unlocked_Filtered.csv"
df = pd.read_csv(input_review_file)
df.head(5)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,1,4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Product Name,Brand Name,Price,Rating,Reviews,Review Votes
0,Apple 64GB iPhone 4S White (Factory Unlocked),NaN,234.98,5.0,I love my iPhone 4S. It arrived completely sea...,1.0
1,Apple 64GB iPhone 4S White (Factory Unlocked),NaN,234.98,5.0,Very nice Items. Fits my needs and very fast s...,1.0
2,Apple 64GB iPhone 4S White (Factory Unlocked),NaN,234.98,5.0,very good phone,0.0
3,Apple 64GB iPhone 4S White (Factory Unlocked),NaN,234.98,5.0,thank you good,0.0
4,Apple 64GB iPhone 4S White (Factory Unlocked),NaN,234.98,5.0,Very good phones worth what I paid,0.0


In [0]:
GroupedReviews = df.groupby('Product Name')['Reviews'].apply(list)


In [0]:
len(GroupedReviews)

700

In [0]:
GroupedReviews[[60]]

Product Name
Apple iPhone 5 16GB (White) - T-Mobile    [Works great with t-mobile., The phone works p...
Name: Reviews, dtype: object

In [0]:
GroupedReviews[10]

['Shouldn\'t be advertised as "UNLOCKED" NOT compatible with Sprint. FML.',
 "Very surprised with the quality since I didn't expect much! Its really in a very good condition and works perfectly! Very happy with the purchase.",
 'very good i like a product',
 "Just recently bought this phone, I started using it and people can't hear me, and the keyboard doesn't function,",
 'ok',
 'It was as advertised',
 'Received as expected. Much better service than the Pantech phone. Like the Hot Spot availability to use with other devices when needed.',
 'The iPhone itself appeared to be near new. However, the sim tool and ear buds were missing, even though the copy clear states that all original accessories were included. Vendor will not provide.',
 'Very good',
 'Terrific product',
 'Excellent Product',
 "it wasn't unlocked it was Verizon",
 'Phone not unlock',
 'nice product! delivered in the time!',
 'Good job .',
 "Though the product is good but the battery worked pathetic. I had to charge the

In [0]:
ProductNumber = 100

In [0]:
ProductNumber = 200
#remove some stop words 
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
  
# example_sent = "This is a sample sentence, showing off the stop words filtration."
  
stop_words = nltk.corpus.stopwords.words('english')
newStopWords = ['iphone','5s','iPhone']
stop_words.extend(newStopWords)
  
word_tokens = word_tokenize(' '.join(str(v) for v in GroupedReviews[ProductNumber])) 
  
filtered_sentence = [w for w in word_tokens if not w in stop_words] 
# " ".join(filtered_sentence)

CondensedReview = ' '.join(str(v) for v in filtered_sentence)
# CondensedReview = ' '.join(str(v) for v in GroupedReviews[ProductNumber])

from rake_nltk import Rake

# Uses stopwords for english from NLTK, and all puntuation characters by
# default
r = Rake()
# Extraction given the text.
r.extract_keywords_from_text(CondensedReview)
print (GroupedReviews[[ProductNumber]])
[eachTuple for eachTuple in r.get_ranked_phrases_with_scores() if len(eachTuple[1].split())<5 and len(eachTuple[1].split())>1]


Product Name
Apple iPhone 6 128GB Factory Unlocked GSM Smartphone w/ 12MP Camera - Gold (Certified Refurbished)    [This phone came quickly, looked new, no scrat...
Name: Reviews, dtype: object


[(29.835714285714282, 'bought refurbished last one'),
 (27.419047619047618, 'used one month n'),
 (26.216666666666665, 'got 6s working perfectly'),
 (25.2, 'last long'),
 (24.56989247311828, 'phone came like new'),
 (24.51036866359447, 'great phone working perfectly'),
 (24.328703703703702, 'looks works like new'),
 (24.303225806451614, 'turned phone worked fine'),
 (24.2, 'order came 3 days'),
 (22.833333333333332, 'also use south korea'),
 (21.903225806451612, 'cell phone perfect conditions'),
 (21.836559139784946, 'concerned ordering refurbished phone'),
 (21.814285714285717, 'also better option would'),
 (21.375, 'looks brand new'),
 (21.33073903396484, 'phone runs great works'),
 (21.21666666666667, 'gift camera broken n'),
 (21.017511520737326, 'great phone great price'),
 (21.0, 'comes cracked screen'),
 (20.875, 'get activate receive calls'),
 (20.833333333333332, 'came box charger'),
 (20.76989247311828, 'got phone time'),
 (20.652380952380955, 'want one verizon network'),
 (2

In [0]:
from textblob import TextBlob

In [0]:
ReviewTopicPolarity = [(eachTuple[1],TextBlob(eachTuple[1]).sentiment.polarity) for eachTuple in r.get_ranked_phrases_with_scores() if len(eachTuple[1].split())<5 and len(eachTuple[1].split())>1 ]
ReviewTopicPolarity

[('bought refurbished last one', 0.0),
 ('used one month n', 0.0),
 ('got 6s working perfectly', 1.0),
 ('last long', -0.025),
 ('phone came like new', 0.13636363636363635),
 ('great phone working perfectly', 0.9),
 ('looks works like new', 0.13636363636363635),
 ('turned phone worked fine', 0.4166666666666667),
 ('order came 3 days', 0.0),
 ('also use south korea', 0.0),
 ('cell phone perfect conditions', 1.0),
 ('concerned ordering refurbished phone', 0.0),
 ('also better option would', 0.5),
 ('looks brand new', 0.13636363636363635),
 ('phone runs great works', 0.8),
 ('gift camera broken n', -0.4),
 ('great phone great price', 0.8),
 ('comes cracked screen', 0.0),
 ('get activate receive calls', 0.0),
 ('came box charger', 0.0),
 ('got phone time', 0.0),
 ('want one verizon network', 0.0),
 ('product perfect arrived time', 1.0),
 ('great condition worked great', 0.8),
 ('scratched apple emblem defects', 0.0),
 ('power button wo n', 0.0),
 ('little nervous ordering phone', -0.1875),

In [0]:
# Postive Poliarites 
[eachTuple[0] for eachTuple in ReviewTopicPolarity if eachTuple[1]>0]

['got 6s working perfectly',
 'phone came like new',
 'great phone working perfectly',
 'looks works like new',
 'turned phone worked fine',
 'cell phone perfect conditions',
 'also better option would',
 'looks brand new',
 'phone runs great works',
 'great phone great price',
 'product perfect arrived time',
 'great condition worked great',
 'camera works fine',
 'fast delivery great product',
 'daughter happy pleased ...',
 'even great deal',
 'good price also',
 'good condition ... nice',
 'clearer website carrier originally',
 'satisfied purchase great price',
 'everything fine love',
 'home button working ok',
 'arrives fast mint condition',
 'far everything else works',
 'color new ipad gold',
 'lovvvee extremely pleased purchase',
 'review aware',
 'straight talk good connection',
 'came great condition',
 'phone perfect condition',
 'everything phone great',
 'definitely worth money',
 'lightly tapped 30 seconds',
 'working great price',
 'phone came quickly',
 'love great con

In [0]:
# Negtive Polarity
[eachTuple[0] for eachTuple in ReviewTopicPolarity if eachTuple[1]<0]

['last long',
 'gift camera broken n',
 'little nervous ordering phone',
 'another waste 2 hours',
 'thought something still wrong',
 'sides minor medium scratches',
 'worst part phone n',
 'expected refurbished product',
 'skeptical phone',
 'could expected',
 'waste money',
 'worst part',
 'quicker expected',
 'small blemishes',
 'visible annoying']

In [0]:
# Spell Corrected 
[(TextBlob(eachTuple[1]).correct(),TextBlob(eachTuple[1]).correct().sentiment) for eachTuple in r.get_ranked_phrases_with_scores() if len(eachTuple[1].split())<4 ]

[(TextBlob("come him card"), Sentiment(polarity=0.0, subjectivity=0.0)),
 (TextBlob("recently became responsive"),
  Sentiment(polarity=0.0, subjectivity=0.25)),
 (TextBlob("`` '' features"), Sentiment(polarity=0.0, subjectivity=0.0)),
 (TextBlob("in sure wont"),
  Sentiment(polarity=0.5, subjectivity=0.8888888888888888)),
 (TextBlob("seems working fine"),
  Sentiment(polarity=0.4166666666666667, subjectivity=0.5)),
 (TextBlob("middle screen unable"),
  Sentiment(polarity=-0.25, subjectivity=0.25)),
 (TextBlob("set service easily"),
  Sentiment(polarity=0.43333333333333335, subjectivity=0.8333333333333334)),
 (TextBlob("phone week yet"), Sentiment(polarity=0.0, subjectivity=0.0)),
 (TextBlob("love problems captivating"),
  Sentiment(polarity=0.5, subjectivity=0.8)),
 (TextBlob("looks almost new"),
  Sentiment(polarity=0.13636363636363635, subjectivity=0.45454545454545453)),
 (TextBlob("took phone local"), Sentiment(polarity=0.0, subjectivity=0.0)),
 (TextBlob("store hour away"), Sentim

In [0]:
len(r.get_ranked_phrases_with_scores()[0][1].split())

8

In [0]:
[eachTuple for eachTuple in r.get_ranked_phrases_with_scores() if len(eachTuple[1].split())<4 ]

[(12.0, 'work correctly ...'),
 (9.0, 'start unless wants'),
 (9.0, 'often drop service'),
 (9.0, 'call numbers randomly'),
 (8.833333333333334, 'turn wo n'),
 (8.5, 'loved first came'),
 (8.333333333333334, 'started drop calls'),
 (8.2, 'far good'),
 (8.0, 'poor coverage'),
 (8.0, 'brand new'),
 (7.0, 'exellent condition'),
 (6.0, 'phone n'),
 (5.5, 'product brand'),
 (4.5, 'loved 4s'),
 (4.333333333333334, 'started using'),
 (4.333333333333334, 'started action'),
 (4.333333333333334, 'ca n'),
 (4.0, 'must say'),
 (4.0, 'make switch'),
 (4.0, 'gps way'),
 (4.0, 'get lte'),
 (4.0, 'efficiently delivered'),
 (4.0, 'buy refurbished'),
 (1.0, 'pleased'),
 (1.0, 'much')]

Product Name
Apple iPhone 5C Pink 32GB Unlocked GSM Smartphone (Certified Refurbished)    [Very weak wifi range. Cannot see Wi-Fi access...
Name: Reviews, dtype: object


[(109.66666666666667,
  'nine phone bad quality sound barley works screen blanked first day'),
 (45.266666666666666, 'brought good dropped screen came phone excellent'),
 (34.0, 'wouldnt know daughter lost 3days purchase'),
 (31.5, '5c blue chargers port wo n'),
 (28.0, 'makes loud vibrating sound switched'),
 (25.0, 'guys need check phones shipping'),
 (24.0, 'came 2 days like suppose'),
 (23.2, 'good price good condition opened'),
 (16.166666666666668, 'recommend anyone order phone'),
 (16.0, 'switching silent mode side'),
 (16.0, 'fi access points unless'),
 (15.5, 'looks fine packaged nicely'),
 (15.0, 'honestly looked brand new'),
 (13.266666666666666, 'good phone price'),
 (10.6, 'everything good cracks'),
 (9.5, 'would totally recommend'),
 (9.5, 'looks really great'),
 (9.0, 'weak wifi range'),
 (9.0, 'pod everything telephone'),
 (6.666666666666667, 'received phone'),
 (6.0, 'return 5c'),
 (6.0, 'let know'),
 (5.0, 'new shape'),
 (4.666666666666667, 'phone'),
 (4.0, 'see wi'),

In [0]:
Review = """Getting familiar with smartphone.
So far so good. The phone works like new. It didn't have any visible scratches or anything. I'm very happy :)
Ordered the phone and waited a week for it to get to my home and not even cut on ðŸ˜¡ðŸ˜¡ NEVER again save your money
had phone for 4 months so far...everything is still working good
Perfect!
I shure did order for iPhone 6
Perfect!
no quality
battery doesn't last and the phone randomly restarts. Definitely will be getting my money back.
apple iphone 5S 16gb WORKING 2 HOURS
Very good although it took two more days than planed to get it because of the season.
It works great. It hasn't overheated or the battery isn't damaged like the other reviews said. I was scared to buy it at first because of the reviews but it hasn't had any problems.
The phone was certified reburbished but was defective, battery heats up tremendously, very slow, fingerprint login did not work, the cable that came for charging the phone was also defective, it had a slight annoying sound when charging, like a buzz. Unfortunately I bought the phone while on vacations in the US and did not try it until I was back in my country until some weeks later when I could not return it. I sent an email to the seller telling them about the problem, as I had to be in the US for some days to see if I could return it or exchange it, but the "return window" was closed, and even though I explained the situation, that I was not in the country to send it back, etc. they did not care. Do yourself a favor and do not buy refurbished phones, at least not from this seller.
works as expected, for my daughter he is not complaining
Works like a charm. As good as new.
I was very happy to find this iPhone 5S for such a good price, since it had been certified refurbished. I bought it a few months ago and it worked wonderfully. However I started noticing issues with the sound after about two months. I could only use the speaker function to dial and to receive calls. If I used earbuds, it seemed to work better. However, yesterday when I put my earbuds in to listen to music, no sound came out at all. When I tried to answer a phone cal, I put it on speaker and could barely understand anything my friend was saying. She couldn't understand me either.I'm pretty disappointed with the quality of this phone and wish that these issues had occurred within a month so I could return this phone and get my money back. Unfortunately it's too late now and I'm stuck with a phone that can't call or receive calls and can't even act as an iPod for music. Pretty disappointing indeed.
I ordered this phone and when I received it it won't charge up or cut on.
Got it from Techarvard. Broken charging cable and charging port of phone. Returned it.
This phone states it is AT&T compatible and unlocked but AT&T could not activate the phone, said that Verizon would have to unlock it.I purchased 8 for a business and probably will have to return them all. Called the seller and they gave me a song and dance abouthow I could unlock the phone. If it is advertised unlocked, it should be unlocked.
Works great. Exactly what I needed!
"""

In [0]:
# Convert to list
data = df.Reviews.values.tolist()
data[:2]

['I love my iPhone 4S. It arrived completely sealed in the box and it is brand new. Thank you.',
 'Very nice Items. Fits my needs and very fast shipping. A++++ Seller.']

In [0]:
#Switch 
data = Review

In [0]:
data

'Getting familiar with smartphone.\nSo far so good. The phone works like new. It didn\'t have any visible scratches or anything. I\'m very happy :)\nOrdered the phone and waited a week for it to get to my home and not even cut on ðŸ˜¡ðŸ˜¡ NEVER again save your money\nhad phone for 4 months so far...everything is still working good\nPerfect!\nI shure did order for iPhone 6\nPerfect!\nno quality\nbattery doesn\'t last and the phone randomly restarts. Definitely will be getting my money back.\napple iphone 5S 16gb WORKING 2 HOURS\nVery good although it took two more days than planed to get it because of the season.\nIt works great. It hasn\'t overheated or the battery isn\'t damaged like the other reviews said. I was scared to buy it at first because of the reviews but it hasn\'t had any problems.\nThe phone was certified reburbished but was defective, battery heats up tremendously, very slow, fingerprint login did not work, the cable that came for charging the phone was also defective, i

In [0]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words)

[[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [],

In [0]:
data_words = data

In [0]:
data_words

'Getting familiar with smartphone.\nSo far so good. The phone works like new. It didn\'t have any visible scratches or anything. I\'m very happy :)\nOrdered the phone and waited a week for it to get to my home and not even cut on ðŸ˜¡ðŸ˜¡ NEVER again save your money\nhad phone for 4 months so far...everything is still working good\nPerfect!\nI shure did order for iPhone 6\nPerfect!\nno quality\nbattery doesn\'t last and the phone randomly restarts. Definitely will be getting my money back.\napple iphone 5S 16gb WORKING 2 HOURS\nVery good although it took two more days than planed to get it because of the season.\nIt works great. It hasn\'t overheated or the battery isn\'t damaged like the other reviews said. I was scared to buy it at first because of the reviews but it hasn\'t had any problems.\nThe phone was certified reburbished but was defective, battery heats up tremendously, very slow, fingerprint login did not work, the cable that came for charging the phone was also defective, i

In [0]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words]])

['G', 'e', 't', 't', 'i', 'n', 'g', ' ', 'f', 'a', 'm', 'i', 'l', 'i', 'a', 'r', ' ', 'w', 'i', 't', 'h', ' ', 's', 'm', 'a', 'r', 't', 'p', 'h', 'o', 'n', 'e', '.', '\n', 'S', 'o', ' ', 'f', 'a', 'r', ' ', 's', 'o', ' ', 'g', 'o', 'o', 'd', '.', ' ', 'T', 'h', 'e', ' ', 'p', 'h', 'o', 'n', 'e', ' ', 'w', 'o', 'r', 'k', 's', ' ', 'l', 'i', 'k', 'e', ' ', 'n', 'e', 'w', '.', ' ', 'I', 't', ' ', 'd', 'i', 'd', 'n', "'", 't', ' ', 'h', 'a', 'v', 'e', ' ', 'a', 'n', 'y', ' ', 'v', 'i', 's', 'i', 'b', 'l', 'e', ' ', 's', 'c', 'r', 'a', 't', 'c', 'h', 'e', 's', ' ', 'o', 'r', ' ', 'a', 'n', 'y', 't', 'h', 'i', 'n', 'g', '.', ' ', 'I', "'", 'm', ' ', 'v', 'e', 'r', 'y', ' ', 'h', 'a', 'p', 'p', 'y', ' ', ':', ')', '\n', 'O', 'r', 'd', 'e', 'r', 'e', 'd', ' ', 't', 'h', 'e', ' ', 'p', 'h', 'o', 'n', 'e', ' ', 'a', 'n', 'd', ' ', 'w', 'a', 'i', 't', 'e', 'd', ' ', 'a', ' ', 'w', 'e', 'e', 'k', ' ', 'f', 'o', 'r', ' ', 'i', 't', ' ', 't', 'o', ' ', 'g', 'e', 't', ' ', 't', 'o', ' ', 'm', 'y', ' 

/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [0]:
trigram

In [0]:
!pip install rake-nltk

  Running setup.py bdist_wheel for rake-nltk ... - \ | / done
  Stored in directory: /root/.cache/pip/wheels/ef/92/fc/271b3709e71a96ffe934b27818946b795ac6b9b8ff8682483f
Successfully built rake-nltk


In [0]:
from rake_nltk import Rake

# Uses stopwords for english from NLTK, and all puntuation characters by
# default
r = Rake()

# Extraction given the text.
r.extract_keywords_from_text(Review)

# Extraction given the list of strings where each string is a sentence.
# r.extract_keywords_from_sentences(data)

# To get keyword phrases ranked highest to lowest.
r.get_ranked_phrases()

# To get keyword phrases ranked highest to lowest with scores.
r.get_ranked_phrases_with_scores()

[(42.0, 'apple iphone 5s 16gb working 2 hours'),
 (25.0, 'ðÿ ˜¡ ðÿ ˜¡ never'),
 (15.25, 'still working good perfect'),
 (11.25, 'could barely understand anything'),
 (10.5, 'iphone 6 perfect'),
 (9.9, 'phone works like new'),
 (8.5, 'pretty disappointing indeed'),
 (8.5, 'iphone 5s'),
 (8.4, 'phone randomly restarts'),
 (8.0, 'started noticing issues'),
 (8.0, 'slight annoying sound'),
 (8.0, 'far ... everything'),
 (8.0, 'complaining works like'),
 (7.5, 'buy refurbished phones'),
 (7.0, 'happy :) ordered'),
 (7.0, 'broken charging cable'),
 (4.5, 'works great'),
 (4.5, 'pretty disappointed'),
 (4.5, 'damaged like'),
 (4.5, 'certified refurbished'),
 (4.4, 'phone states'),
 (4.4, 'phone cal'),
 (4.25, 'good price'),
 (4.25, 'good although'),
 (4.0, 'worked wonderfully'),
 (4.0, 'weeks later'),
 (4.0, 'visible scratches'),
 (4.0, 'verizon would'),
 (4.0, 'two months'),
 (4.0, 'took two'),
 (4.0, 'receive calls'),
 (4.0, 'purchased 8'),
 (4.0, 'occurred within'),
 (4.0, 'months ago'),
 